## Data Generation

In [30]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from myPackage.my_module import *
from torchvision.transforms import ToTensor

In [231]:
def data_to_bins(data):
    counts=np.zeros(100)
    for dat in data:
        try:
            counts[int(dat*100)]+=1/len(data)
        except IndexError:
            pass
    return counts

def projection(matrix):
    return np.real(np.matrix([[matrix[0,0],0,0,matrix[0,3]/2+matrix[3,0]/2],\
                                [0,matrix[2,2]/2+matrix[1,1]/2,0,0],\
                                [0,0,matrix[2,2]/2+matrix[1,1]/2,0],\
                                [matrix[0,3]/2+matrix[3,0]/2,0,0,matrix[3,3]]]))
qubit_switch = np.matrix([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])
    
def phase_rotation(phi=np.random.random()*2*pi):
    from scipy.linalg import expm
    sig_z = np.array([[1,0],[0,-1]], dtype='complex')
    U = tens_prod2d(expm((1j * phi * sig_z)), expm((-1j * phi * sig_z)))
    return U

def phase_rotation_dag(phi=np.random.random()*2*pi):
    return np.transpose(np.conjugate(phase_rotation(phi)))

proj_bins=[]
for i in range(10000):
    initial_matrix = rand_PSDM(4)
    projected = projection(initial_matrix)
    parameters = [projected[0,0], projected[0,3], projected[1,1], projected[3,3]]
    proj_data=[]
    for j in range(2000):
        phi=np.random.random()*2*pi
        rotated = phase_rotation(phi)@initial_matrix@phase_rotation_dag(phi)
        switched = qubit_switch@rotated@qubit_switch
        proj_data.append(np.trace(np.real(switched@np.outer(ZeroZero,ZeroZero))))
    
    proj_bins.append(data_to_bins(proj_data))
    
    if (i+1)%10 == 0:
        print(f'{i+1}/10000 done')
    

10/10000 done
20/10000 done
30/10000 done
40/10000 done
50/10000 done
60/10000 done
70/10000 done
80/10000 done
90/10000 done
100/10000 done
110/10000 done
120/10000 done
130/10000 done
140/10000 done
150/10000 done
160/10000 done
170/10000 done
180/10000 done
190/10000 done
200/10000 done
210/10000 done
220/10000 done
230/10000 done
240/10000 done
250/10000 done
260/10000 done
270/10000 done
280/10000 done
290/10000 done
300/10000 done
310/10000 done
320/10000 done
330/10000 done
340/10000 done
350/10000 done
360/10000 done
370/10000 done
380/10000 done
390/10000 done
400/10000 done
410/10000 done
420/10000 done
430/10000 done
440/10000 done
450/10000 done
460/10000 done
470/10000 done
480/10000 done
490/10000 done
500/10000 done
510/10000 done
520/10000 done
530/10000 done
540/10000 done
550/10000 done
560/10000 done
570/10000 done
580/10000 done
590/10000 done
600/10000 done
610/10000 done
620/10000 done
630/10000 done
640/10000 done
650/10000 done
660/10000 done
670/10000 done
680/

In [229]:
proj_data

[0.24129063356890973]

In [232]:
import pickle
with open('ML_int_proj.dat', 'wb') as file:
    pickle.dump(proj_bins, file)

In [5]:
def projection(matrix):
    return np.real(np.matrix([[matrix[0,0],0,0,matrix[0,3]/2+matrix[3,0]/2],\
                                [0,matrix[2,2]/2+matrix[1,1]/2,0,0],\
                                [0,0,matrix[2,2]/2+matrix[1,1]/2,0],\
                                [matrix[0,3]/2+matrix[3,0]/2,0,0,matrix[3,3]]]))
samples = []
for i in range(10000):
    initial_matrix = rand_PSDM(4)
    projected = projection(initial_matrix)
    parameters = [projected[0,0], projected[0,3], projected[1,1], projected[3,3]]
    projected = density_matrix(projected)
    projected.set(20000)
    samples.append((projected.data, parameters))
    if (i+1)%10 == 0:
        print(f'{i+1}/10000 done')
    

10/10000 done
20/10000 done
30/10000 done
40/10000 done
50/10000 done
60/10000 done
70/10000 done
80/10000 done
90/10000 done
100/10000 done
110/10000 done
120/10000 done
130/10000 done
140/10000 done
150/10000 done
160/10000 done
170/10000 done
180/10000 done
190/10000 done
200/10000 done
210/10000 done
220/10000 done
230/10000 done
240/10000 done
250/10000 done
260/10000 done
270/10000 done
280/10000 done
290/10000 done
300/10000 done
310/10000 done
320/10000 done
330/10000 done
340/10000 done
350/10000 done
360/10000 done
370/10000 done
380/10000 done
390/10000 done
400/10000 done
410/10000 done
420/10000 done
430/10000 done
440/10000 done
450/10000 done
460/10000 done
470/10000 done
480/10000 done
490/10000 done
500/10000 done
510/10000 done
520/10000 done
530/10000 done
540/10000 done
550/10000 done
560/10000 done
570/10000 done
580/10000 done
590/10000 done
600/10000 done
610/10000 done
620/10000 done
630/10000 done
640/10000 done
650/10000 done
660/10000 done
670/10000 done
680/

In [70]:
samples_new = []
for hist, par in samples_load:
    samples_new.append((data_to_bins(hist), par))

In [73]:
import pickle
with open('ML_proj_samps.dat', 'wb') as file:
    pickle.dump(samples_new, file)

In [8]:
import pickle
with open('ML_proj_samps.dat', 'rb') as file:
    samples_load = pickle.load(file)

In [74]:
samples_train = samples_new[:8000]
samples_val = samples_new[8000:]

In [44]:
hist, params = samples_load[0]
hist.astype('double')

array([0.24111, 0.20476, 0.2126 , ..., 0.24117, 0.20626, 0.24237])

In [60]:
class proj_dataset(Dataset):
    
    def __init__(self,samples):
        super(Dataset, self).__init__()
        self.samples = samples
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        hist, params = self.samples[idx]
        hist = torch.from_numpy(hist).float()
        params = torch.Tensor(params).float()
        return hist, params

In [180]:
class Net(nn.Module):
    
    def __init__(self, input=100, output=4):
        super().__init__()
        
        self.input_layer = nn.Linear(input, 50)
        self.output_layer = nn.Linear(20, output)
        self.hidden_layers = nn.Sequential(
            nn.Linear(50, 100),
            nn.Sigmoid(),
            nn.Dropout(p=0.5),
            nn.Linear(100,25),
            nn.Sigmoid(),
            nn.Dropout(p=0.3),
            nn.Linear(25,20),
            nn.Sigmoid(),
            nn.Dropout(p=0.3),
        )
    
    def forward(self, x):
        x = self.input_layer(x)
        x = self.hidden_layers(x)
        x = self.output_layer(x)
        return x   
        

In [204]:
class WeightedLoss(nn.Module):
    
    def __init__(self):
        super(WeightedLoss, self).__init__()
        self.weights=torch.Tensor([0.1,0.1,0.7,0.1])
    
    def forward(self, inputs, targets):
            loss = (inputs - targets) * (inputs - targets) @ self.weights
            return loss.mean()

In [212]:
#Trainig & validation loop
def fit(model, samples_train, samples_val, batch_size=10, lr=0.05, epochs=10, dataset = proj_dataset, criterion=nn.MSELoss()):    
    from torchmetrics import MeanSquaredError
    data_loader_train = DataLoader(dataset(samples_train), shuffle=True, batch_size=batch_size)
    data_loader_val = DataLoader(dataset(samples_val), shuffle=True, batch_size=batch_size)
    optimizer = torch.optim.RMSprop(model.parameters(), lr=lr, weight_decay=0.5) 
    metrics = MeanSquaredError()

    for epoch in range(epochs):
        #Training
        model.train()
        for hist, params in data_loader_train:
            optimizer.zero_grad()
            pred = model(hist)
            loss = criterion(pred, params)
            loss.backward()
            optimizer.step()
            metrics(pred, params)
        train_loss = metrics.compute()
        metrics.reset()
        #Validation
        model.eval()
        with torch.no_grad():
            for hist, params in data_loader_val:
                pred = model(hist)
                metrics(pred, params)
        val_loss = metrics.compute()
        metrics.reset()
        
        print(f'Epoch {epoch+1}/{epochs}: training loss: {train_loss:.5f}, validation loss: {val_loss:.5f}')
    
    return model    
    
             

In [139]:
# Evaluation

def evaluate_model(model, samples_eval, batch_size=10, dataset=proj_dataset):
    import torchmetrics
    metrics = torchmetrics.MeanSquaredError()
    data_loader = DataLoader(dataset(samples_eval), shuffle=True, batch_size=batch_size)
    model.eval()
    with torch.no_grad():
        for hist, params in data_loader:
            pred = model(hist)
            metrics(pred, params)
    return metrics.compute()    


In [220]:
model = fit(Net(), proj_bins[:8000], proj_bins[8000:], batch_size=10, lr=0.05, epochs=10, criterion=WeightedLoss())


ValueError: too many values to unpack (expected 2)

In [145]:
torch.save(model.state_dict(), 'proj_model.model')

In [216]:
model.eval()
model(torch.from_numpy(samples_new[96][0]).float())

tensor([0.1960, 0.1308, 0.2028, 0.1806], grad_fn=<ViewBackward0>)

In [197]:
for hist, par in DataLoader(proj_dataset(samples_new), shuffle=True, batch_size=100):
    print(par)

tensor([[ 0.3354, -0.0165,  0.2870,  0.0906],
        [ 0.1889,  0.1396,  0.2780,  0.2550],
        [ 0.2649, -0.0952,  0.2593,  0.2165],
        [ 0.3346,  0.1355,  0.2659,  0.1336],
        [ 0.2448,  0.0404,  0.2107,  0.3338],
        [ 0.2261,  0.0901,  0.2509,  0.2720],
        [ 0.2780, -0.0081,  0.1859,  0.3502],
        [ 0.2004,  0.0356,  0.3035,  0.1926],
        [ 0.0726,  0.0909,  0.1815,  0.5643],
        [ 0.1588,  0.0086,  0.3895,  0.0622],
        [ 0.1085, -0.0889,  0.2715,  0.3486],
        [ 0.1369, -0.0270,  0.3763,  0.1106],
        [ 0.1680,  0.0632,  0.3033,  0.2254],
        [ 0.2600, -0.1931,  0.1839,  0.3721],
        [ 0.5549, -0.0605,  0.1861,  0.0728],
        [ 0.1296, -0.0247,  0.3332,  0.2040],
        [ 0.1877,  0.0090,  0.3501,  0.1121],
        [ 0.2586,  0.0662,  0.3212,  0.0991],
        [ 0.2098,  0.0021,  0.3063,  0.1775],
        [ 0.1006, -0.0119,  0.4091,  0.0812],
        [ 0.2724,  0.0700,  0.3349,  0.0577],
        [ 0.3466, -0.0640,  0.2956

In [ ]:
model_loaded = Net()
model_loaded.load_state_dict(torch.load('proj_model.model'))

In [160]:

torch.cuda.is_available()


False

## Test with Werner states

In [106]:
df = pd.read_csv('werner_sample.csv', index_col='Unnamed: 0')
df_test = pd.read_csv('werner_test.csv', index_col='Unnamed: 0')
df.head()

,Angle,Visibility,"[0.0, 0.01]","[0.01, 0.02]","[0.02, 0.03]","[0.03, 0.04]","[0.04, 0.05]","[0.05, 0.06]","[0.06, 0.07]","[0.07, 0.08]",...,"[0.9, 0.91]","[0.91, 0.92]","[0.92, 0.93]","[0.93, 0.94]","[0.94, 0.95]","[0.95, 0.96]","[0.96, 0.97]","[0.97, 0.98]","[0.98, 0.99]","[0.99, 1.0]"
0,0.333026,0.045424,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.568860,0.448408,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.620751,0.267946,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.127500,0.635826,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.401182,0.933977,0.0,0.00898,0.02606,0.02698,0.02618,0.02702,0.02636,0.0261,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [137]:
def df_to_wer_data(df):
    wer_data=[]
    for i in range(len(df)):
        wer_data.append((df.iloc[i].values[2:], df.iloc[i].values[:2]))
    return wer_data 

In [115]:
class werner_dataset(Dataset):
    
    def __init__(self, data, transform = torch.from_numpy):
        super(Dataset, self).__init__()
        self.data = data
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        hist, params = self.data[idx]
        hist = self.transform(hist).float()
        params = self.transform(params).float()
        return hist, params

In [127]:
class WerNet(nn.Module):
    
    def __init__(self, input=100, output=4):
        super().__init__()
        
        self.input_layer = nn.Linear(input, 95)
        self.output_layer = nn.Linear(95, output)
        self.hidden_layers = nn.Sequential(
            nn.Sigmoid(),
            nn.Linear(95, 95),
            nn.Sigmoid(),
            ##nn.Dropout(p=0.5),
            nn.Linear(95,95),
            nn.Sigmoid(),

        )
    
    def forward(self, x):
        x = self.input_layer(x)
        x = self.hidden_layers(x)
        x = self.output_layer(x)
        return x   
        

In [ ]:
model_wer = fit(WerNet(input=100, output=2), wer_data[:8000], wer_data[8000:], dataset = werner_dataset, epochs=10)

In [136]:
torch.save(model_wer.state_dict(), 'model_wer.model')

In [162]:
model_loaded = WerNet(input=100, output=2)
model_loaded.load_state_dict(torch.load('model_wer.model'))

WerNet(
  (input_layer): Linear(in_features=100, out_features=95, bias=True)
  (output_layer): Linear(in_features=95, out_features=2, bias=True)
  (hidden_layers): Sequential(
    (0): Sigmoid()
    (1): Linear(in_features=95, out_features=95, bias=True)
    (2): Sigmoid()
    (3): Linear(in_features=95, out_features=95, bias=True)
    (4): Sigmoid()
  )
)

In [140]:
df_test = pd.read_csv('werner_test.csv', index_col='Unnamed: 0')
wer_test = df_to_wer_data(df_test)
evaluate_model(model_wer, wer_test, dataset=werner_dataset)

tensor(0.0018)

In [163]:
model_wer(torch.from_numpy(wer_test[10][0]).float())

tensor([0.6590, 0.3315], grad_fn=<ViewBackward0>)

In [158]:
wer_test[10][1]

array([0.65972, 0.32552])